# 构建模型

一旦数据收集标注完毕,衡量指标选择完成.接下来要进行模型的开发了.

许多资料都认为构建模型是机器学习项目唯一的步骤而跳过了问题定义和数据收集.认为这两个过程已经由其他人完成.但是显然这里作者表达了不同意见.构建模型只是机器学习项目的一环,而且还不是最难的一环.(最难的是创建数据集).


## 准备数据

通过前面的例子,我们应该清楚,模型处理的数据通常并不是原始数据.我们需要进行预处理,将原始数据转换成更适合神经网络的格式.这个过程有

- 张量化(矢量化)
- 规范化
- 处理缺失值

其实我们在前面的例子中已经实践过这些步骤了,这里理论上回顾一下.


### 张量化

神经网络所有的数据都是典型的带浮点数的张量,任何输入神经网络的数据都要转换成张量输入.

imdb 数据集的例子中,我们将词频前 10000 的单词序列转换成了 32 位浮点数的张量.

当然如果数据已经是张量了,这个步骤可用跳过.


### 规范化

mnist 的例子中,每个像素是 0-255 的整数,我们将其转换成了 float32 / 255.最终范围是 0-1 的浮点数.

波士顿房价预测例子中,有非常多的特征,这些特征的类型不同,有整数有浮点数,相同类型的取值范围相差又相当大.最终我们是将所有特征都进行了规范化,使其标准差为 1,均值为 0.

一般而言,取值较大的数据或者内部差异很大的数据(两个特征,一个取值范围是0-1,另一个是 100-200)输入神经网络并不安全.这样会引发大的梯度更新,阻碍网络收敛.

适合输入神经网络的数据特征

- 取值范围较小,通常是 0-1.
- 同质化,不同特征的取值范围大致相同.

在此之上也可以更加严格的同一化,每个特征单独粗规范化.

- 均值为 0.
- 标准差为 1.

更加严格的同一化并不是必须的,需要按照问题分析.(mnist 显然没必要)

严格均一化显然 numpy 非常适合.

```py
x -= x.mean(axis=0)
x /= x.std(axis=0)
```


### 处理缺失值

实际收集的样本中,某些类型数据缺失是常有的事情.波士顿房价预测中人均犯罪率就不是所有样本都存在.

缺失值的样本超多,那可以选择放弃这个特征.但是还有其他处理方式.

- 如果这个特征是分类,那么数值缺失也可以算是一种类别.增加一个新的类别-数据缺失.
- 如果特征是数字类型,要避免将数据缺失填充成一个任意的值(例如 0 1),这样的任意值可能会在特征潜在的流行空间产生不连续,阻碍模型拟合.可以选择使用这个特征的平均值或中位数填充.或者相反可以训练另一个模型,使用样本的其他特征来预测缺失值.

还有一种情况是测试集存在缺失值,但是模型是在完整的训练集训练的.这种情况下,模型不会忽略缺失值.这种情况下需要人为的产生带缺失值的训练样本.

- 将一些训练样本复制若干次.
- 在一些批次放弃测试集中可能出现缺失值的特征.
